In [1]:
import openpyxl
from openai import OpenAI

client = OpenAI(api_key="")
import os
import time
import json

def get_completion(prompt, model="gpt-3.5-turbo-1106", retries=3):
    for _ in range(retries):
        try:
            messages = [{"role": "system", "content": "You are designed to output the final answer in JSON."},
                        {"role": "user", "content": prompt}
                       ]
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0.7,
                response_format={ "type": "json_object" }
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"An error occurred: {e}. Retrying...")
            time.sleep(2)
    return None

wb_path = 'C:/Work/Articles/Chat-GPT/specializeddoc/Osteoarthritis/Part1.Base model evaluation/3.5/treatment_based_3.5.xlsx'
wb = openpyxl.load_workbook(wb_path)
sheet = wb['Sheet1']


try:
    with open("checkpoint.json", "r") as f:
        checkpoint = json.load(f)
    start_row = checkpoint["row"]
    print(f"Resuming from row {start_row}...")
except FileNotFoundError:
    start_row = 1

prompts = [
     """
       Provide answer for the following question, the answer should be based on well acknowledged published clinical guidelines: 

       The answer should concisely state the if the treatment is recommended, if any, the strength of this recommendation (if specified), and the level of certainty of the evidence supporting it (if specified). 
       The answer should provide brief explanation and additional information worth mentioning.
       
       For example: 'Treatment X is [recommended/not recommended/may be used] for condition Y. The strength of recommendation is [recommendation strength], and the certainty of evidence is [evidence level].
                    Brief explanation and additional information is [Brief explanation and additional information]
       
       
       Note: Output in the example json format. 
       Example in plain words: AA for osteoarthritis. Recommendation status: EE The strength of recommendation is BB The certainty of evidence is CC. According to DD
                               Brief explanation and additional information:xxxxxx
       Example in json format：Intervention: AA ;RecommendationStatus: EE;StrengthOfRecommendation: BB;LevelOfCertainty: CC；Source: DD
                               Brief explanation and additional information:xxxxxx
       
       The question is: 

""",
    """You are a digital osteoarthritis specialists, leveraging its professional knowledge to provide insights on osteoarthritis based on clinical guidelines. You avoids speculation and sticks to facts. If a query cannot be answered, you will state not known.
       
       Provide answer for the following question, the answer should be based on well acknowledged published clinical guidelines:
       
       The answer should concisely state the if the treatment is recommended, if any, the strength of this recommendation (if specified), and the level of certainty of the evidence supporting it (if specified). 
       The answer should provide brief explanation and additional information worth mentioning.
       
       Note: Output in the example json format. 
       Example in plain words: AA for osteoarthritis. Recommendation status: EE The strength of recommendation is BB The certainty of evidence is CC. According to DD
                               Brief explanation and additional information:xxxxxx
       Example in json format：Intervention: AA ;RecommendationStatus: EE;StrengthOfRecommendation: BB;LevelOfCertainty: CC；Source: DD
                               Brief explanation and additional information:xxxxxx
       Perform the task step by step.
       

"""
]

stop_script = False  # Flag to indicate if we should stop the script

for content_row in range(start_row, len(sheet['F']) + 1):
    if stop_script:
        break

    col_index = 7
    content = sheet.cell(row=content_row, column=6).value
    print(f"Processing row {content_row}...")
    
    for prompt in prompts:
        for run in range(5):
            full_prompt = f"{prompt} {content}"
            response = get_completion(full_prompt)
            
            
            if response is None:
                with open("checkpoint.json", "w") as f:
                    json.dump({"row": content_row}, f)
                print(f"Stopping at row {content_row}. Checkpoint saved.")
                stop_script = True  # Set the flag to stop the script
                break

            if sheet.cell(row=content_row, column=col_index).value is None:  # Do not overwrite existing content
                sheet.cell(row=content_row, column=col_index).value = response.strip()
            
            col_index += 1

    wb.save('C:/Work/Articles/Chat-GPT/specializeddoc/Osteoarthritis/Part1.Base model evaluation/3.5/treatment_based_3.5.xlsx')
    print(f"Saved data for row {content_row}")

    if os.path.exists("checkpoint.json"):
        os.remove("checkpoint.json")

wb.save('C:/Work/Articles/Chat-GPT/specializeddoc/Osteoarthritis/Part1.Base model evaluation/3.5/treatment_based_3.5.xlsx')
print("The task has been successfully completed.")


Processing row 1...
Saved data for row 1
Processing row 2...
Saved data for row 2
Processing row 3...
Saved data for row 3
Processing row 4...
Saved data for row 4
Processing row 5...
Saved data for row 5
Processing row 6...
Saved data for row 6
Processing row 7...
Saved data for row 7
Processing row 8...
Saved data for row 8
Processing row 9...
Saved data for row 9
Processing row 10...
Saved data for row 10
Processing row 11...
Saved data for row 11
Processing row 12...
Saved data for row 12
Processing row 13...
Saved data for row 13
Processing row 14...
Saved data for row 14
Processing row 15...
Saved data for row 15
Processing row 16...
Saved data for row 16
Processing row 17...
Saved data for row 17
Processing row 18...
Saved data for row 18
Processing row 19...
Saved data for row 19
Processing row 20...
Saved data for row 20
Processing row 21...
Saved data for row 21
Processing row 22...
Saved data for row 22
Processing row 23...
Saved data for row 23
Processing row 24...
Saved da

In [ ]:
,
    """ 
    You are 3 digital osteoarthritis specialists,  leveraging professional knowledge to provide insights on osteoarthritis based on clinical guidelines. You avoids speculation and sticks to facts. If a query cannot be answered, you will state not known.
 
    Step 1. Make sure each expert understand the meaning of the question.

    Step 2. Each expert will independently give out an answer to the question, the answer should be based on well acknowledged published clinical guidelines: The answer should concisely state the if the treatment is recommended, if any, the strength of this recommendation (if specified), and the level of certainty of the evidence supporting it (if specified). For example: 'Treatment X is [recommended/not recommended/may be used] for condition Y. The strength of recommendation is [recommendation strength], and the certainty of evidence is [evidence level].
   
    Step 3. Three experts will discuss the answers provided in step 3, and choose the best answer.

    Step 4. After discussing, the final answer will be provided.
    
       The final answer should concisely state the if the treatment is recommended, if any, the strength of this recommendation (if specified), and the level of certainty of the evidence supporting it (if specified). 
       The final answer should provide brief explanation and additional information worth mentioning.
       
      
       Note: Output in the example json format. 
       Example in plain words: AA for osteoarthritis. Recommendation status: EE The strength of recommendation is BB The certainty of evidence is CC. According to DD
                               Brief explanation and additional information:xxxxxx
       Example in json format：Intervention: AA ;RecommendationStatus: EE;StrengthOfRecommendation: BB;LevelOfCertainty: CC；Source: DD
                               Brief explanation and additional information:xxxxxx
    
    The question is:
 """,
    """
    You are 3 digital osteoarthritis specialists, , leveraging professional knowledge to provide insights on osteoarthritis based on clinical guidelines. You avoids speculation and sticks to facts. If a query cannot be answered, you will state not known.
    
    The three specialists finish the task by performing the following steps sequentially, and show me the thoughts step by step:
    
    Step 1. Make sure each expert understand the meaning of the question.

    Step 2. Each expert will independently give out an answer to the question, the answer should be based on well acknowledged published clinical guidelines: The answer should concisely state the if the treatment is recommended, if any, the strength of this recommendation (if specified), and the level of certainty of the evidence supporting it (if specified). For example: 'Treatment X is [recommended/not recommended/may be used] for condition Y. The strength of recommendation is [recommendation strength], and the certainty of evidence is [evidence level].
   
    Step 3. Each expert will share and discuss their thought process in detail, taking into account the previous thoughts of others and admitting any errors.

    Step 4. Post-discussion, independently revise answers if necessary:
    
    Step 5: Iterative Refinement Loop, revisit Step 1 with the revised diagnostic opinions and any new information. Cycle through Steps 2-6 again, incorporating previous findings.
            Continue this iterative process until consensus of the final answer is reached.
    
       The final answer should concisely state the if the treatment is recommended, if any, the strength of this recommendation (if specified), and the level of certainty of the evidence supporting it (if specified). 
       The final answer should provide brief explanation and additional information worth mentioning.
       
      
       Note: Output in the example json format. 
       Example in plain words: AA for osteoarthritis. Recommendation status: EE The strength of recommendation is BB The certainty of evidence is CC. According to DD
                               Brief explanation and additional information:xxxxxx
       Example in json format：Intervention: AA ;RecommendationStatus: EE;StrengthOfRecommendation: BB;LevelOfCertainty: CC；Source: DD
                               Brief explanation and additional information:xxxxxx
    
       The question is:

    """